In [5]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
os.chdir("/home/edgan/projects/sketchstore/python")
DATA_DIR = "/mnt/disks/data/datasets/"

In [3]:
def sample_until_lazy(xs, size, seed=0):
    r = np.random.RandomState(seed)
    i = 0
    res = set()
    while len(res) < size and len(xs) > 0:
        new_x = r.choice(xs)
        res.add(new_x)
        i += 1
        if i % 1000 == 0:
            xs = xs[np.isin(xs, res, invert=True)]
    return list(res)

# Zipf Big

In [ ]:
fname = os.path.join(DATA_DIR, "zipfbig/zipf10M.csv")
tname = os.path.join(DATA_DIR, "zipfbig/zipf10M-xtrack.csv")

In [10]:
r = np.random.RandomState(seed=0)
chunk_size = 10_000_000
n_chunks = 5
n_max = 100_000_000
for i in tqdm(range(n_chunks)):
    x_stream = r.zipf(1.1, size=2*chunk_size)
    x_stream = x_stream[x_stream < n_max][:chunk_size]
    use_header = False
    if i == 0:
        use_header = True
    fname = os.path.join(DATA_DIR, "zipfbig/zipf-{}.csv".format(i))
    pd.DataFrame({"x": x_stream}).to_csv(fname, index=False, header=use_header)

# Simple Uniform

In [12]:
xtrack = np.linspace(0, 1, 101)
df_xtrack = pd.DataFrame({"x_track": xtrack})
fname = os.path.join(DATA_DIR, "uniform/uniform-xtrack.csv")
df_xtrack.to_csv(fname, index=False)

In [13]:
r = np.random.RandomState(0)
xvals = r.uniform(0,1,size=1_000_000)
df_xvals = pd.DataFrame({"x": xvals})
fname = os.path.join(DATA_DIR, "uniform/uniform.csv")
df_xvals.to_csv(fname, index=False)

# Cube

In [3]:
import testdata.bench_gen

In [12]:
df, dim_names = testdata.bench_gen.gen_data(
    10_000_000,
    [(10, 1),
     (10, 1),
     (10, 1),
     (10, 1),
     ],
    f_skew=1.1,
    f_card=50000,
    seed=0,
)

In [19]:
fname = os.path.join(DATA_DIR, "sketchstore_synth/bcube4_10M.feather")
tfname = os.path.join(DATA_DIR, "sketchstore_synth/bcube4_10M_f_track.csv")
tqname = os.path.join(DATA_DIR, "sketchstore_synth/bcube4_10M_q_track.csv")

In [ ]:
df.to_feather(fname)

In [17]:
df = pd.read_feather(fname)

In [20]:
x_track = sample_until_lazy(df["f"], size=200, seed=0)
df_track = pd.DataFrame({
    "x_track": x_track,
})
df_track.to_csv(
    tfname, 
    index=False,
)

In [21]:
x_track = np.percentile(df["q"], np.arange(0,101))
df_track = pd.DataFrame({
    "x_track": x_track,
})
df_track.to_csv(
    tqname, 
    index=False,
)